Name: Tanaya Pawar Netid : tp2623

Data Science Bootcamp assignment 3

In [8]:
import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()


In [9]:
# Creating SALES table
cursor.execute('''
CREATE TABLE SALES (
    Date TEXT,
    Order_id INTEGER,
    Item_id INTEGER,
    Customer_id INTEGER,
    Quantity INTEGER,
    Revenue REAL
)
''')

# Creating ITEMS table
cursor.execute('''
CREATE TABLE ITEMS (
    Item_id INTEGER,
    Item_name TEXT,
    Price REAL,
    department TEXT
)
''')

# Creating CUSTOMERS table
cursor.execute('''
CREATE TABLE CUSTOMERS (
    Customer_id INTEGER,
    First_name TEXT,
    Last_name TEXT,
    Address TEXT
)
''')


In [10]:
# Insert data into SALES table
cursor.executemany('''
INSERT INTO SALES (Date, Order_id, Item_id, Customer_id, Quantity, Revenue) VALUES (?, ?, ?, ?, ?, ?)
''', [
    ('2023-03-18', 1, 1, 1, 2, 200),
    ('2023-03-18', 2, 2, 2, 1, 100),
    ('2023-01-15', 3, 3, 1, 3, 300)
])

# Insert data into ITEMS table
cursor.executemany('''
INSERT INTO ITEMS (Item_id, Item_name, Price, department) VALUES (?, ?, ?, ?)
''', [
    (1, 'Item A', 100.0, 'Electronics'),
    (2, 'Item B', 100.0, 'Books'),
    (3, 'Item C', 100.0, 'Clothing')
])

# Insert data into CUSTOMERS table
cursor.executemany('''
INSERT INTO CUSTOMERS (Customer_id, First_name, Last_name, Address) VALUES (?, ?, ?, ?)
''', [
    (1, 'John', 'Doe', '123 Elm St'),
    (2, 'Jane', 'Smith', '456 Oak St')
])

# Commit changes to the database
conn.commit()


In [11]:
# Query for total orders on March 18th
cursor.execute('SELECT COUNT(DISTINCT Order_id) AS total_orders FROM SALES WHERE Date = "2023-03-18"')
total_orders = cursor.fetchone()[0]
print("Total Orders on March 18th:", total_orders)





Total Orders on March 18th: 2


In [12]:
# Query for John Doe orders on 18th March 2023
cursor.execute('''
SELECT COUNT(DISTINCT s.Order_id) AS john_doe_orders
FROM SALES s
JOIN CUSTOMERS c ON s.Customer_id = c.Customer_id
WHERE s.Date = "2023-03-18" AND c.First_name = "John" AND c.Last_name = "Doe"
''')
john_doe_orders = cursor.fetchone()[0]
print("John Doe Orders on March 18th:", john_doe_orders)


John Doe Orders on March 18th: 1


In [13]:
# Query for total customers who purchased in January 2023 and their average spend
cursor.execute('''
SELECT COUNT(DISTINCT Customer_id) AS total_customers, AVG(total_spend) AS avg_spend_per_customer
FROM (
    SELECT Customer_id, SUM(Revenue) AS total_spend
    FROM SALES
    WHERE Date BETWEEN "2023-01-01" AND "2023-01-31"
    GROUP BY Customer_id
) AS customer_totals
''')
january_customers = cursor.fetchone()
print("January Customers:", january_customers[0], "Average Spend:", january_customers[1])

January Customers: 1 Average Spend: 300.0


In [14]:
# Query departments generating less than $600 in 2022
query_low_revenue_departments = '''
SELECT i.department, SUM(s.Revenue) AS total_revenue
FROM SALES s
JOIN ITEMS i ON s.Item_id = i.Item_id
WHERE strftime('%Y', s.Date) = '2022'
GROUP BY i.department
HAVING total_revenue < 600;
'''
cursor.execute(query_low_revenue_departments)
low_revenue_departments = cursor.fetchall()
# Display departments with less than $600 in revenue in 2022
print("Departments with Less Than $600 in Revenue in 2022:")
for dept in low_revenue_departments:
    print(dept[0], "with revenue", dept[1])



Departments with Less Than $600 in Revenue in 2022:


In [16]:
# Query for maximum and minimum revenue from an order
query_revenue_range = '''
SELECT MAX(Revenue) AS max_revenue, MIN(Revenue) AS min_revenue
FROM SALES;
'''
cursor.execute(query_revenue_range)
revenue_range = cursor.fetchone()
# Display the most and least revenue from an order
print("Maximum Revenue from an Order:", revenue_range[0])
print("Minimum Revenue from an Order:", revenue_range[1])


Maximum Revenue from an Order: 300.0
Minimum Revenue from an Order: 100.0


In [17]:
# Query for details of the most lucrative order
query_most_lucrative_order = '''
SELECT *
FROM SALES
WHERE Revenue = (SELECT MAX(Revenue) FROM SALES);
'''
cursor.execute(query_most_lucrative_order)
most_lucrative_orders = cursor.fetchall()
# Display details of the most lucrative order
print("Details of the Most Lucrative Order:")
for order in most_lucrative_orders:
    print("Order ID:", order[1], "Item ID:", order[2], "Customer ID:", order[3], "Quantity:", order[4], "Revenue:", order[5])


Details of the Most Lucrative Order:
Order ID: 3 Item ID: 3 Customer ID: 1 Quantity: 3 Revenue: 300.0
